In [56]:
import pandas as pd
import os

class PowerPlants(object):
    
    def __init__(self):
        self.database_file = 'database.csv'
        # Simple country name mapping
        self.country_map = {
            'FR': 'France',
            'GB': 'Great Britain'
        }

    def _get_full_country_name(self, country_code_or_name):
        """
        Converts a country identifier (code or name) to a standardized full name.
        """
        return self.country_map.get(str(country_code_or_name).upper())
    
    def load_new_data_from_file(self, file_path: str) -> pd.DataFrame:
        """
        Loads and processes data from a specified CSV file.
        Handles missing data, maps country codes, and adds metadata.
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Input file not found: {file_path}")
            
        df = pd.read_csv(file_path)

        rename_map = {
            'Date': 'date',
            'Site Name': 'SiteName',
            'Country': 'country'
        }
        df.rename(columns=lambda c: rename_map.get(c, c.strip()), inplace=True)

        if 'country' in df.columns:
            df['country'] = df['country'].apply(self._get_full_country_name)
        else:
            if 'country' not in df.columns:
                 df['country'] = "Unknown"

        # Date Processing
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], dayfirst=True).dt.strftime('%Y-%m-%d')
        else:
            raise ValueError(f"Date column is missing in the input file: {file_path}")

        if 'Technology' not in df.columns:
            if 'wind_plants.csv' in file_path:
                df['Technology'] = 'Wind'
            elif 'gas_plants.csv' in file_path or 'gas_fr_plants.csv' in file_path:
                df['Technology'] = 'Gas'
            else:
                df['Technology'] = 'Unknown'

        # Volume Processing
        if 'Volume' in df.columns:
            df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce').fillna(0)

        # Add Metadata Columns
        df['updatetime'] = pd.Timestamp.now(tz=None) # Timezone naive timestamp
        df['updatedby'] = 'petroineos' # As per example in image

        # Final Column Selection and Order based on the example output for get_data_from_database
        final_columns_order = ['country', 'date', 'SiteName', 'Technology', 'updatedby', 'updatetime', 'Volume']

        df = df[final_columns_order]
        return df

    def save_new_data(self, input_data: pd.DataFrame):
        """
        Saves the loaded and processed DataFrame to the database CSV file.
        Appends if the file exists, creates with header if it doesn't.
        """
        if not isinstance(input_data, pd.DataFrame):
            raise TypeError("input_data must be a pandas DataFrame.")

        file_exists = os.path.exists(self.database_file)
        input_data.to_csv(self.database_file, mode='a', header=not file_exists, index=False)

    def _read_database(self) -> pd.DataFrame:
        """
        Helper function to read the database CSV file.
        Handles file not existing, empty file, and ensures basic column types.
        """
        expected_cols = ['country', 'date', 'SiteName', 'Technology', 'updatedby', 'updatetime', 'Volume']
        
        if not os.path.exists(self.database_file):
            return pd.DataFrame(columns=expected_cols)
        
        try:
            db_df = pd.read_csv(self.database_file)
            if db_df.empty and not list(db_df.columns): # File exists but is truly empty or only headers of empty table
                 return pd.DataFrame(columns=expected_cols)
        except pd.errors.EmptyDataError: # File is completely empty (no content at all)
            return pd.DataFrame(columns=expected_cols)

        # Ensure essential columns exist, fill with NA/default if not
        for col in expected_cols:
            if col not in db_df.columns:
                if col == 'Volume':
                    db_df[col] = 0.0
                elif col in ['date', 'updatetime']:
                    db_df[col] = pd.NaT
                else:
                    db_df[col] = pd.NA # Use pandas NA for general missing string/object data

        # Select only expected columns in the defined order
        db_df = db_df[expected_cols]

        # Convert data types, coercing errors
        db_df['date'] = pd.to_datetime(db_df['date'], errors='coerce')
        db_df['updatetime'] = pd.to_datetime(db_df['updatetime'], errors='coerce')
        db_df['Volume'] = pd.to_numeric(db_df['Volume'], errors='coerce').fillna(0)
        
        # Ensure string columns are strings, to handle potential all-NA columns read as float etc.
        for col in ['country', 'SiteName', 'Technology', 'updatedby']:
            if col in db_df.columns:
                 db_df[col] = db_df[col].astype(str).replace('nan', pd.NA).replace('<NA>', pd.NA)

        return db_df

    def get_data_from_database(self) -> pd.DataFrame:
        """
        Returns the most recently updated data for every symbol (SiteName, date combination)
        from the database.
        """
        db_df = self._read_database()
        
        if db_df.empty or 'updatetime' not in db_df.columns or 'SiteName' not in db_df.columns or 'date' not in db_df.columns:
            # Return empty df with correct columns if db is empty or key columns missing
             expected_cols = ['country', 'date', 'SiteName', 'Technology', 'updatedby', 'updatetime', 'Volume']
             return pd.DataFrame(columns=expected_cols)

        # Remove rows where key identifiers or updatetime are NaT/NaN after conversion
        db_df.dropna(subset=['SiteName', 'date', 'updatetime'], inplace=True)
        if db_df.empty:
            expected_cols = ['country', 'date', 'SiteName', 'Technology', 'updatedby', 'updatetime', 'Volume']
            return pd.DataFrame(columns=expected_cols)

        # Sort by SiteName, date, and then by updatetime descending to get the latest entry first
        # for each (SiteName, date) group.
        latest_df = db_df.sort_values(by=['SiteName', 'date', 'updatetime'], ascending=[True, True, False])
        
        # Drop duplicates based on (SiteName, date), keeping the first occurrence (which is the most recent)
        result_df = latest_df.drop_duplicates(subset=['SiteName', 'date'], keep='first').copy()
        
        # Format columns as per the example output
        result_df['date'] = pd.to_datetime(result_df['date']).dt.strftime('%Y-%m-%d')
        result_df['Volume'] = result_df['Volume'].apply(lambda x: f"{x:.6f}")
        
        # Ensure final column order
        final_columns_order = ['country', 'date', 'SiteName', 'Technology', 'updatedby', 'updatetime', 'Volume']
        result_df = result_df[final_columns_order]
        
        return result_df

    def aggregate_data_to_monthly(self) -> pd.DataFrame:
        """
        Returns a DataFrame with monthly average, min, and max Volume for each plant.
        The format matches the structure seen in 'month_average_min_max.png'. [cite: 15]
        """
        current_data = self.get_data_from_database()
        
        if current_data.empty:
            return pd.DataFrame() # Return empty DataFrame if no data

        # Convert Volume back to numeric and date to datetime for calculations
        current_data['Volume'] = pd.to_numeric(current_data['Volume'], errors='coerce')
        current_data['date'] = pd.to_datetime(current_data['date'], errors='coerce')
        
        current_data.dropna(subset=['Volume', 'date', 'SiteName'], inplace=True)
        if current_data.empty:
            return pd.DataFrame()

        # Group by SiteName and month (first day of month), then aggregate
        # Using a dictionary for agg ensures column names 'Min', 'Mean', 'Max'
        monthly_agg = current_data.groupby(
            ['SiteName', pd.Grouper(key='date', freq='MS')] 
        )['Volume'].agg(Min='min', Mean='mean', Max='max')
        
        if monthly_agg.empty:
            return pd.DataFrame()

        # Unstack 'SiteName' to transform it into column levels
        result_df = monthly_agg.unstack(level='SiteName')
        
        # Swap levels to get (SiteName, Stat) for easier sorting and final naming
        result_df = result_df.swaplevel(0, 1, axis=1)
        
        # Sort columns: first by SiteName (alphabetically), then by Stat (Min, Mean, Max order)
        result_df = result_df.sort_index(axis=1, level=0, sort_remaining=False) 
                                       
        # Flatten the multi-level column names to 'SiteName Stat' format
        result_df.columns = [f"{site} {stat}" for site, stat in result_df.columns]
        
        # Format the index (date) to 'YYYY-MM-DD' string
        result_df.index = result_df.index.strftime('%Y-%m-%d')
        
        return result_df

    def aggregate_data_to_country(self) -> pd.DataFrame:
        """
        Returns a DataFrame with the total power production (Volume) by country and technology type.
        Matches structure of 'country_sum.png'. [cite: 18]
        """
        current_data = self.get_data_from_database()
        
        if current_data.empty:
            return pd.DataFrame(columns=['country', 'Technology', 'Volume'])

        # Convert Volume back to numeric for calculation
        current_data['Volume'] = pd.to_numeric(current_data['Volume'], errors='coerce')
        current_data.dropna(subset=['Volume', 'country', 'Technology'], inplace=True)

        if current_data.empty:
            return pd.DataFrame(columns=['country', 'Technology', 'Volume'])

        # Group by 'country' and 'Technology', then sum 'Volume'
        country_agg_df = current_data.groupby(['country', 'Technology'])['Volume'].sum().reset_index()
        
        return country_agg_df

In [57]:
# Initialize a PowerPLants object
pp = PowerPlants()

# Load and save data sequentially as per instructions
new_data_wind = pp.load_new_data_from_file('wind_plants.csv')
pp.save_new_data(new_data_wind)

new_data_gas = pp.load_new_data_from_file('gas_plants.csv')
pp.save_new_data(new_data_gas)

new_data_gas_fr = pp.load_new_data_from_file('gas_fr_plants.csv')
pp.save_new_data(new_data_gas_fr)

In [58]:
print("Most Recently Updated Data (get_data_from_database)")
latest_data = pp.get_data_from_database()
print(latest_data)

Most Recently Updated Data (get_data_from_database)
            country        date    SiteName Technology   updatedby  \
2394         France  2024-01-01    Blenod-5        Gas  petroineos   
2395         France  2024-01-02    Blenod-5        Gas  petroineos   
2396         France  2024-01-03    Blenod-5        Gas  petroineos   
2397         France  2024-01-04    Blenod-5        Gas  petroineos   
2398         France  2024-01-05    Blenod-5        Gas  petroineos   
...             ...         ...         ...        ...         ...   
3826  Great Britain  2025-04-21  Pembroke-2        Gas  petroineos   
3827  Great Britain  2025-04-22  Pembroke-2        Gas  petroineos   
3828  Great Britain  2025-04-23  Pembroke-2        Gas  petroineos   
3829  Great Britain  2025-04-24  Pembroke-2        Gas  petroineos   
3830  Great Britain  2025-04-25  Pembroke-2        Gas  petroineos   

                     updatetime       Volume  
2394 2025-05-29 10:39:35.435734  6753.000000  
2395 2025-05-

In [59]:
print("Monthly Aggregated Data (aggregate_data_to_monthly) ---")
monthly_data = pp.aggregate_data_to_monthly()
print(monthly_data)
print("\n")

Monthly Aggregated Data (aggregate_data_to_monthly) ---
            Blenod-5 Min  Blenod-5 Mean  Blenod-5 Max  Hornsea-1 Min  \
date                                                                   
2024-01-01        3295.0    5198.806452        6890.0      44.937929   
2024-02-01        3489.0    4889.896552        6895.0       9.973714   
2024-03-01        3122.0    4752.774194        6399.0      34.240333   
2024-04-01        3235.0    4988.466667        6970.0      67.202179   
2024-05-01        3059.0    5012.032258        6898.0      14.319806   
2024-06-01        3083.0    4928.666667        6870.0      16.042355   
2024-07-01        3115.0    4933.677419        6680.0       8.136290   
2024-08-01        3104.0    5296.258065        6973.0       6.695016   
2024-09-01        3110.0    4858.066667        6922.0      24.260159   
2024-10-01        3061.0    5365.160000        6973.0      75.917168   
2024-11-01        3108.0    4912.733333        6942.0      10.153305   
2024-12-

In [61]:
print("Country Aggregated Data (aggregate_data_to_country)")
country_data = pp.aggregate_data_to_country()
print(country_data)

Country Aggregated Data (aggregate_data_to_country)
         country Technology        Volume
0         France        Gas  2.379583e+06
1  Great Britain        Gas  6.768124e+06
2  Great Britain      Wind   4.753565e+05
